In [ ]:
import numpy as np
from scipy import ndimage as ndi
from scipy import signal
from lenspack import peaks
from lenspack.peaks import find_peaks2d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd
import sys
import os

In [ ]:
sp_val = '{}/sp_validation/sp_validation'.format(os.environ['HOME'])

for sc in ['survey', 'params', 'io','basic']:
    script = os.path.join(sp_val, sc)
    %run $script

# Define the data we need

In [ ]:
# Load the R matrix obtain on the different size of calibration
# R selection
R_selec_1deg = np.load('./P3X/var_save/dm_0_013/R_selec_1deg.npy')
R_selec_2deg = np.load('./P3X/var_save/dm_0_013/R_selec_2deg.npy')
R_selec_4deg = np.load('./P3X/var_save/dm_0_013/R_selec_4deg.npy')
R_selec_05deg = np.load('./P3X/var_save/dm_0_013/R_selec_05deg.npy')
R_selec_025deg = np.load('./P3X/var_save/dm_0_013/R_selec_025deg.npy')

# Rshear
R_shear_1deg = np.load('./P3X/var_save/dm_0_013/R_shear_1deg.npy')
R_shear_2deg = np.load('./P3X/var_save/dm_0_013/R_shear_2deg.npy')
R_shear_4deg = np.load('./P3X/var_save/dm_0_013/R_shear_4deg.npy')
R_shear_05deg = np.load('./P3X/var_save/dm_0_013/R_shear_05deg.npy')
R_shear_025deg = np.load('./P3X/var_save/dm_0_013/R_shear_025deg.npy')

In [ ]:
# Size in degree of the P3 catalogue
size_deg_x = 36
size_deg_y = 20

In [ ]:
def compute_R_real_std(npix_x,npix_y,factor,R):
    # Compute std of R with jackknife spatial
    
    R_real_std = np.zeros((2,2,npix_y,npix_x))

    for i in range(npix_y):
        for j in range(npix_x):
            if (np.sum(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[0,0,i,j] = jackknif_weighted_average(np.ravel(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[0,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[0,0,i,j] = np.nan
                
            if (np.sum(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[1,1,i,j] = jackknif_weighted_average(np.ravel(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[1,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[1,1,i,j] = np.nan
                
            if (np.sum(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[0,1,i,j] = jackknif_weighted_average(np.ravel(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[0,1,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[0,1,i,j] = np.nan
                
            if (np.sum(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])!=0):
                R_real_std[1,0,i,j] = jackknif_weighted_average(np.ravel(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)]), np.ones_like(np.ravel(R[1,0,factor*i:factor*(i+1),factor*j:factor*(j+1)])))[1]
            else:
                R_real_std[1,0,i,j] = np.nan
                
    return R_real_std

In [ ]:
# Std of R shear from the map on 0.25 square degree
R_shear_real_std_05deg = compute_R_real_std(size_deg_x*2,size_deg_y*2,2,R_shear_025deg)
R_shear_real_std_1deg = compute_R_real_std(size_deg_x,size_deg_y,4,R_shear_025deg)
R_shear_real_std_2deg = compute_R_real_std(int(size_deg_x/2),int(size_deg_y/2),8,R_shear_025deg)
R_shear_real_std_4deg = compute_R_real_std(int(size_deg_x/4),int(size_deg_y/4),16,R_shear_025deg)

In [ ]:
# Std of R shear from the map on 0.25 square degree
R_selec_real_std_05deg = compute_R_real_std(size_deg_x*2,size_deg_y*2,2,R_selec_025deg)
R_selec_real_std_1deg = compute_R_real_std(size_deg_x,size_deg_y,4,R_selec_025deg)
R_selec_real_std_2deg = compute_R_real_std(int(size_deg_x/2),int(size_deg_y/2),8,R_selec_025deg)
R_selec_real_std_4deg = compute_R_real_std(int(size_deg_x/4),int(size_deg_y/4),16,R_selec_025deg)

In [ ]:
# Load the corresponding number of galaxy
ngal_025deg = np.load('./P3X/var_save/ngal_025deg.npy')
ngal_05deg = np.load('./P3X/var_save/ngal_05deg.npy')
ngal_1deg = np.load('./P3X/var_save/ngal_1deg.npy')
ngal_2deg = np.load('./P3X/var_save/ngal_2deg.npy')
ngal_4deg = np.load('./P3X/var_save/ngal_4deg.npy')

# Computation of the errors : < dR/R > or < dR >/< R >

## R shear

In [ ]:
# Component 11
# 1st method : < dR/R >
err_Rshear_05deg_moy = np.mean(R_shear_real_std_05deg[0,0,:,:]/R_shear_05deg[0,0,:,:])
err_Rshear_1deg_moy = np.mean(R_shear_real_std_1deg[0,0,:,:]/R_shear_1deg[0,0,:,:])
err_Rshear_2deg_moy = np.mean(R_shear_real_std_2deg[0,0,:,:]/R_shear_2deg[0,0,:,:])
err_Rshear_4deg_moy = np.mean(R_shear_real_std_4deg[0,0,:,:]/R_shear_4deg[0,0,:,:])

# 2nd method : < dR/R >
err_Rshear_05deg = np.mean(R_shear_real_std_05deg[0,0,:,:])/np.mean(R_shear_05deg[0,0,:,:])
err_Rshear_1deg = np.mean(R_shear_real_std_1deg[0,0,:,:])/np.mean(R_shear_1deg[0,0,:,:])
err_Rshear_2deg = np.mean(R_shear_real_std_2deg[0,0,:,:])/np.mean(R_shear_2deg[0,0,:,:])
err_Rshear_4deg = np.mean(R_shear_real_std_4deg[0,0,:,:])/np.mean(R_shear_4deg[0,0,:,:])

In [ ]:
# Component 22
# 1st method : < dR/R >
err_Rshear_05deg_moy22 = np.mean(R_shear_real_std_05deg[1,1,:,:]/R_shear_05deg[1,1,:,:])
err_Rshear_1deg_moy22 = np.mean(R_shear_real_std_1deg[1,1,:,:]/R_shear_1deg[1,1,:,:])
err_Rshear_2deg_moy22 = np.mean(R_shear_real_std_2deg[1,1,:,:]/R_shear_2deg[1,1,:,:])
err_Rshear_4deg_moy22 = np.mean(R_shear_real_std_4deg[1,1,:,:]/R_shear_4deg[1,1,:,:])

# 2nd method : < dR/R >
err_Rshear_05deg22 = np.mean(R_shear_real_std_05deg[1,1,:,:])/np.mean(R_shear_05deg[1,1,:,:])
err_Rshear_1deg22 = np.mean(R_shear_real_std_1deg[1,1,:,:])/np.mean(R_shear_1deg[1,1,:,:])
err_Rshear_2deg22 = np.mean(R_shear_real_std_2deg[1,1,:,:])/np.mean(R_shear_2deg[1,1,:,:])
err_Rshear_4deg22 = np.mean(R_shear_real_std_4deg[1,1,:,:])/np.mean(R_shear_4deg[1,1,:,:])

In [ ]:
# Definie the value to plot for R shear
x_pos = [0.5,1,2,4]

# Component 11
moy_dr_r_Rshear = [err_Rshear_05deg_moy,err_Rshear_1deg_moy,err_Rshear_2deg_moy,err_Rshear_4deg_moy]
dr_r_Rshear = [err_Rshear_05deg,err_Rshear_1deg,err_Rshear_2deg,err_Rshear_4deg]

# Component 22
moy_dr_r_Rshear22 = [err_Rshear_05deg_moy22,err_Rshear_1deg_moy22,err_Rshear_2deg_moy22,err_Rshear_4deg_moy22]
dr_r_Rshear22 = [err_Rshear_05deg22,err_Rshear_1deg22,err_Rshear_2deg22,err_Rshear_4deg22]

# Component 12 and 21
moy_dr_r_Rshear21 = [np.mean(R_shear_real_std_05deg[1,0,:,:]),np.mean(R_shear_real_std_1deg[1,0,:,:]),np.mean(R_shear_real_std_2deg[1,0,:,:]),np.mean(R_shear_real_std_4deg[1,0,:,:])]
moy_dr_r_Rshear12 = [np.mean(R_shear_real_std_05deg[0,1,:,:]),np.mean(R_shear_real_std_1deg[0,1,:,:]),np.mean(R_shear_real_std_2deg[0,1,:,:]),np.mean(R_shear_real_std_4deg[0,1,:,:])]

In [ ]:
# Plot the 4 components and the 2 methods
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(x_pos,moy_dr_r_Rshear,label='R_11')
plt.plot(x_pos,moy_dr_r_Rshear22,label='R_22')
plt.plot(x_pos,moy_dr_r_Rshear12,label='R_12')
plt.plot(x_pos,moy_dr_r_Rshear21,label='R_21')
plt.legend()
plt.axhline(y=0,color='k')
plt.xlabel('taille calib')
plt.ylabel('< dR_shear/R_shear >')

plt.subplot(122)
plt.plot(x_pos,dr_r_Rshear,label='R_11')
plt.plot(x_pos,dr_r_Rshear22,label='R_22')
plt.plot(x_pos,moy_dr_r_Rshear12,label='R_12')
plt.plot(x_pos,moy_dr_r_Rshear21,label='R_21')
plt.legend()
plt.axhline(y=0,color='k')
plt.xlabel('taille calib')
plt.ylabel('< dR_shear >/< R_shear >')
plt.show()

In [ ]:
# As R_11 ~ R_22 and R_12 ~ R_21, we keep the mean of the diagonal and the mean of the off-diagonal
# The 2 methods are similar so I kepp < dR/R >
moy_Rshear_diag = np.zeros(4)
moy_Rshear_off_diag = np.zeros(4)

for i in range(4):
    moy_Rshear_diag[i] = (moy_dr_r_Rshear[i]+moy_dr_r_Rshear22[i])/2
    moy_Rshear_off_diag[i] = (moy_dr_r_Rshear12[i]+moy_dr_r_Rshear21[i])/2

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(x_pos,moy_Rshear_diag,label=r'$\langle dR_{ii}/R_{ii} \rangle$')
plt.plot(x_pos,moy_Rshear_off_diag,label=r'$R_{ij}$')
plt.legend(fontsize=20)
plt.axhline(y=0,color='k')
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('calibration size (°)',fontsize=20)
plt.ylabel(r'fractional error of $R_{shear}$',fontsize=20)
plt.savefig('./dr_shear.png')
plt.show()

## R selection

In [ ]:
# Component 11
# 1st method : < dR/R >
err_Rselec_05deg_moy = abs(np.mean(R_selec_real_std_05deg[0,0,:,:]/R_selec_05deg[0,0,:,:]))
err_Rselec_1deg_moy = abs(np.mean(R_selec_real_std_1deg[0,0,:,:]/R_selec_1deg[0,0,:,:]))
err_Rselec_2deg_moy = abs(np.mean(R_selec_real_std_2deg[0,0,:,:]/R_selec_2deg[0,0,:,:]))
err_Rselec_4deg_moy = abs(np.mean(R_selec_real_std_4deg[0,0,:,:]/R_selec_4deg[0,0,:,:]))

# 2nd method : < dR/R >
err_Rselec_05deg = abs(np.mean(R_selec_real_std_05deg[0,0,:,:])/np.mean(R_selec_05deg[0,0,:,:]))
err_Rselec_1deg = abs(np.mean(R_selec_real_std_1deg[0,0,:,:])/np.mean(R_selec_1deg[0,0,:,:]))
err_Rselec_2deg = abs(np.mean(R_selec_real_std_2deg[0,0,:,:])/np.mean(R_selec_2deg[0,0,:,:]))
err_Rselec_4deg = abs(np.mean(R_selec_real_std_4deg[0,0,:,:])/np.mean(R_selec_4deg[0,0,:,:]))

In [ ]:
# Component 22
# 1st method : < dR/R >
err_Rselec_05deg_moy22 = abs(np.mean(R_selec_real_std_05deg[1,1,:,:]/R_selec_05deg[1,1,:,:]))
err_Rselec_1deg_moy22 = abs(np.mean(R_selec_real_std_1deg[1,1,:,:]/R_selec_1deg[1,1,:,:]))
err_Rselec_2deg_moy22 = abs(np.mean(R_selec_real_std_2deg[1,1,:,:]/R_selec_2deg[1,1,:,:]))
err_Rselec_4deg_moy22 = abs(np.mean(R_selec_real_std_4deg[1,1,:,:]/R_selec_4deg[1,1,:,:]))

# 2nd method : < dR/R >
err_Rselec_05deg22 = abs(np.mean(R_selec_real_std_05deg[1,1,:,:])/np.mean(R_selec_05deg[1,1,:,:]))
err_Rselec_1deg22 = abs(np.mean(R_selec_real_std_1deg[1,1,:,:])/np.mean(R_selec_1deg[1,1,:,:]))
err_Rselec_2deg22 = abs(np.mean(R_selec_real_std_2deg[1,1,:,:])/np.mean(R_selec_2deg[1,1,:,:]))
err_Rselec_4deg22 = abs(np.mean(R_selec_real_std_4deg[1,1,:,:])/np.mean(R_selec_4deg[1,1,:,:]))

In [ ]:
# Define the value to plot for R selection
# Component 11
moy_dr_r_Rselec = [err_Rselec_05deg_moy,err_Rselec_1deg_moy,err_Rselec_2deg_moy,err_Rselec_4deg_moy]
dr_r_Rselec = [err_Rselec_05deg,err_Rselec_1deg,err_Rselec_2deg,err_Rselec_4deg]

# Component 22
moy_dr_r_Rselec22 = [err_Rselec_05deg_moy22,err_Rselec_1deg_moy22,err_Rselec_2deg_moy22,err_Rselec_4deg_moy22]
dr_r_Rselec22 = [err_Rselec_05deg22,err_Rselec_1deg22,err_Rselec_2deg22,err_Rselec_4deg22]

# Component 12 and 21
moy_dr_r_Rselec21 = [abs(np.mean(R_selec_real_std_05deg[1,0,:,:])),abs(np.mean(R_selec_real_std_1deg[1,0,:,:])),abs(np.mean(R_selec_real_std_2deg[1,0,:,:])),abs(np.mean(R_selec_real_std_4deg[1,0,:,:]))]
moy_dr_r_Rselec12 = [abs(np.mean(R_selec_real_std_05deg[0,1,:,:])),abs(np.mean(R_selec_real_std_1deg[0,1,:,:])),abs(np.mean(R_selec_real_std_2deg[0,1,:,:])),abs(np.mean(R_selec_real_std_4deg[0,1,:,:]))]

In [ ]:
# As R_11 ~ R_22 and R_12 ~ R_21, we keep the mean of the diagonal and the mean of the off-diagonal
# The 2 methods are similar so I kepp < dR/R >
moy_Rselec_diag = np.zeros(4)
moy_Rselec_off_diag = np.zeros(4)

for i in range(4):
    moy_Rselec_diag[i] = (moy_dr_r_Rselec[i]+moy_dr_r_Rselec22[i])/2
    moy_Rselec_off_diag[i] = (moy_dr_r_Rselec12[i]+moy_dr_r_Rselec21[i])/2

In [ ]:
# Plot the 4 components and the 2 methods
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(x_pos,moy_dr_r_Rselec,label='R_11')
plt.plot(x_pos,moy_dr_r_Rselec22,label='R_22')
plt.plot(x_pos,moy_dr_r_Rselec12,label='R_12')
plt.plot(x_pos,moy_dr_r_Rselec21,label='R_21')
plt.legend()
plt.axhline(y=0,color='k')
plt.xlabel('taille calib')
plt.ylabel('< dR_selec/R_selec >')

plt.subplot(122)
plt.plot(x_pos,dr_r_Rselec,label='R_11')
plt.plot(x_pos,dr_r_Rselec22,label='R_22')
plt.plot(x_pos,moy_dr_r_Rselec12,label='R_12')
plt.plot(x_pos,moy_dr_r_Rselec21,label='R_21')
plt.legend()
plt.axhline(y=0,color='k')
plt.xlabel('taille calib')
plt.ylabel('< dR_selec >/< R_selec >')
plt.show()

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(x_pos,moy_Rselec_diag,label=r'$\langle dR_{ii}/R_{ii} \rangle$')
plt.plot(x_pos,moy_Rselec_off_diag,label=r'$R_{ij}$')
plt.legend(fontsize=20)
plt.axhline(y=0,color='k')
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('calibration size (°)',fontsize=20)
plt.ylabel(r'fractional error of $R_{selection}$',fontsize=20)
plt.savefig('./dr_selec.png')
plt.show()

## Additive bias

In [ ]:
# Load the errors of the additive bias obtain on the different calibration size
err_c1_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_1deg.npy')
err_c2_ngmix_1deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_1deg.npy')

err_c1_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_2deg.npy')
err_c2_ngmix_2deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_2deg.npy')

err_c1_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_4deg.npy')
err_c2_ngmix_4deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_4deg.npy')

err_c1_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_05deg.npy')
err_c2_ngmix_05deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_05deg.npy')

err_c1_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/err_c1_ngmix_025deg.npy')
err_c2_ngmix_025deg = np.load('./P3X/var_save/dm_0_013/err_c2_ngmix_025deg.npy')

In [ ]:
# Method : < dc > because c1 ~ 0 and c2 ~ 0
moyc1 = [abs(np.mean(err_c1_ngmix_025deg)),abs(np.mean(err_c1_ngmix_05deg)),abs(np.mean(err_c1_ngmix_1deg)),abs(np.mean(err_c1_ngmix_2deg)),abs(np.mean(err_c1_ngmix_4deg))]
moyc2 = [np.mean(err_c2_ngmix_025deg),np.mean(err_c2_ngmix_05deg),np.mean(err_c2_ngmix_1deg),np.mean(err_c2_ngmix_2deg),np.mean(err_c2_ngmix_4deg)]

In [ ]:
# Definie the position in x
x_pos_c = [0.25,0.5,1,2,4]

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(x_pos_c,moyc1,label=r'$\langle dc_1 \rangle$')
plt.plot(x_pos_c,moyc2,label=r'$\langle dc_2 \rangle$')
plt.axhline(y=0,color='k')
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.legend(fontsize=20)
plt.xlabel('calibration size (°)',fontsize=20)
plt.ylabel(r'fractional error c',fontsize=20)
plt.savefig('./dr_c.png')
plt.show()